# Experiment with the tuner

In [1]:
%reload_ext autoreload
%autoreload 2

# from common.metrics.gradient import Gradient
import torch
from torch.utils.data import DataLoader
import torchinfo
import numpy as np
import pandas as pd
import pytorch_lightning as pl 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from pylab import *
import ast
import os
import sys
from statistics import mean
import warnings

# import modules from ECON model
module_path = os.path.abspath(os.path.join('../tuner/')) # or the path to your source code
sys.path.insert(0, module_path)
from jet_tuner import JetTagger

module_path = os.path.abspath(os.path.join('../../workspace/models/jets/code/')) # or the path to your source code
sys.path.insert(0, module_path)
from jet_datamodule import JetDataModule

# import modules from common metrics
module_path = os.path.abspath(os.path.join('../../workspace/common/metrics/')) # or the path to your source code
sys.path.insert(0, module_path)
from CKA import CKA
from neural_efficiency import NeuralEfficiency


# import modules from common benchmarks
module_path = os.path.abspath(os.path.join('../../workspace/common/benchmarks/')) # or the path to your source code
sys.path.insert(0, module_path)
from noisy_dataset import NoisyDataset


DATA_PATH="/home/jovyan/checkpoint/"    
DATASET_DIR="../../data/JTAG"
DATASET_FILE=DATASET_DIR + "/processed_dataset.h5"

/opt/conda/envs/loss_landscape/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def get_data_module(batch_size, num_workers=12):
    '''
    Method used to get the data modules used during the tests
    '''
    data_module = JetDataModule(
        data_dir=DATASET_DIR,
        data_file=os.path.join(DATASET_DIR, DATASET_FILE),
        batch_size=batch_size,
        num_workers=num_workers)
    
    # checek if we have processed the data
    if not os.path.exists(os.path.join(DATASET_DIR, DATASET_FILE)):
        print('Processing the data...')
        data_module.process_data(save=True)

    data_module.setup(0)
    return data_module

In [3]:
model_path = '../tuner/experiment_models/net_best.pkl'
ne_dataloader = get_data_module(1).test_dataloader()

# load the model
model = JetTagger(
    quantize=(4 < 32),
    precision=[
        4,
        4,
        4+3
    ],
    learning_rate=0.0125,
)
    
# to set the map location
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model(torch.randn((16, 16)))  # Update tensor shapes 
model_param = torch.load(model_path, map_location=device)
model.load_state_dict(model_param['state_dict'])

metric = NeuralEfficiency(model, 
                          ne_dataloader, 
                          performance=0.732,
                          target_layers=['model.dense_1'],
                          max_batches=1000)

result = metric.compute(beta=2)
    
print(result)

Loaded shaped data shape (train): (630000, 16)
Loaded shaped data datatype (train): float32
Loaded shaped data shape (val): (240000, 16)
Loaded shaped data datatype (val): float32
Loading quantized model with bitwidth 4
Computing the Neural efficiency...
5
{'layers_efficiency': {'model.dense_1': 0.7533666774358656}, 'network_efficiency': 0.7533666774358656, 'aIQ': 0.7390540302107835}
